<a href="https://colab.research.google.com/github/harshgupta9723/Quora_question_scrappers/blob/main/Quora_question_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.models import Word2Vec
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from scipy import spatial

import pandas as pd
import numpy as np

# # Downloading the stop words list
nltk.download('stopwords')
# # Loading the stop words in english
stop_words = list(stopwords.words('english'))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
main_data = pd.read_csv('/content/drive/MyDrive/tutree_work/related_q (1).csv')

main_data.head()

In [ ]:
def job_cleaner(text):
    """
            text: a string
            return: cleaned initial string
        """
    text = text.replace("\n", " ")
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords.words('english'))
    text = str(text)
    text = text.lower()  # lowercase text
    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split()
                    if word not in STOPWORDS)  # delete stopwords

    return text

In [ ]:
def make_vector(text):
    vector = []
    vector.append(text.split(" "))
    model = Word2Vec(vector, min_count=1, size=300)
    # list of words
    total_words = list(model.wv.vocab)
    # adding
    one_vector = []
    for i in range(len(total_words)):
        one_vector.append(model[total_words[i]])

    sum_vector = sum(np.asarray(one_vector))
    if (len(total_words) > 0):
        feature_vec = np.divide(sum_vector, len(total_words))

    return feature_vec

In [ ]:
question_list = main_data['Questions']

vectors = []

for question in question_list:
    clean_text = job_cleaner(question)
    vector = make_vector(clean_text)
    vectors.append(vector)

In [ ]:
def find_similarity_score():

    list_of_similar_ques = []

    for question in question_list:
        clean_text = job_cleaner(question)
        vector = make_vector(clean_text)

        similar_question = []

        for i, ques in enumerate(question_list):
            clean_text = job_cleaner(ques)
            q_vector = make_vector(clean_text)

            sim = 1 - spatial.distance.cosine(vector, q_vector)
            if sim > 0.60:
                similar_question.append(ques)

        list_of_similar_ques.append(similar_question)

    return list_of_similar_ques
    
final_list = find_similarity_score()

In [ ]:
final_data = pd.DataFrame()
final_data['question'] = question_list
final_data['similar_question'] = final_list

final_data.to_csv("/content/drive/MyDrive/tutree_work/similar_questions.csv")